<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import tensorflow as tf
import tensorflow.compat.v1.keras.backend as K
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
K.set_session(session)

import numpy as np
import pandas as pd
from skimage import io as sio
from scipy import io

from keras import layers
from keras import Model
from keras.optimizers import Adam
from keras.utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet50

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
ADNI = io.loadmat("ADNI_new.mat")
X, y = ADNI['X'][:,:,:,13:16], ADNI['y']
X = X / np.max(X)
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=42)
io.savemat("ADNI_test.mat", {'y_train' : y_train, 'X_train' : X_train, 
                             'y_test' : y_test, 'X_test' : X_test})

In [3]:
image_input = layers.Input([100, 100, 3])
base_model = ResNet50(weights='imagenet', include_top=False, 
                      pooling='max', input_tensor=image_input)
x = base_model.output
out = layers.Dense(3, activation='softmax', name='output_layer')(x)
model = Model(inputs=image_input, outputs=out)
model.summary()

94773248/94765736 [==============================] - 20s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
__________________________

Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
__________________________________________________________________________________________________


In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs=10, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/10
896/896 [==============================] - 80s 90ms/step - loss: 6.7794 - accuracy: 0.4196 - val_loss: 14.8305 - val_accuracy: 0.2812
Epoch 2/10
896/896 [==============================] - 72s 81ms/step - loss: 6.0415 - accuracy: 0.5290 - val_loss: 359.2986 - val_accuracy: 0.2812
Epoch 3/10
896/896 [==============================] - 69s 77ms/step - loss: 2.8838 - accuracy: 0.6585 - val_loss: 278.4541 - val_accuracy: 0.2812
Epoch 4/10
896/896 [==============================] - 71s 79ms/step - loss: 2.5990 - accuracy: 0.6696 - val_loss: 831.4937 - val_accuracy: 0.3884
Epoch 5/10
896/896 [==============================] - 69s 77ms/step - loss: 1.4948 - accuracy: 0.7143 - val_loss: 266962.4682 - val_accuracy: 0.3884
Epoch 6/10
896/896 [==============================] - 70s 78ms/step - loss: 1.4659 - accuracy: 0.7578 - val_loss: 1141748.2790 - val_accuracy: 0.3884
Epoch 7/10
896/896 [==============================] - 69s 77ms/step - lo

In [5]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.3607142857142857

In [13]:
y_test_pred

array([[0.01271223, 0.00306264, 0.98422515],
       [0.01249018, 0.00312204, 0.98438776],
       [0.01301413, 0.00309779, 0.9838881 ],
       [0.01206621, 0.00306557, 0.9848682 ],
       [0.01394314, 0.00314327, 0.98291355],
       [0.01226588, 0.00295231, 0.98478186],
       [0.01366814, 0.00304374, 0.9832881 ],
       [0.01262902, 0.00304947, 0.9843215 ],
       [0.01246031, 0.00304587, 0.9844938 ],
       [0.01284688, 0.00306842, 0.9840847 ],
       [0.0128315 , 0.00312442, 0.98404413],
       [0.01258329, 0.00305579, 0.984361  ],
       [0.01292625, 0.0030639 , 0.9840098 ],
       [0.01228492, 0.00303646, 0.9846786 ],
       [0.01280362, 0.00309012, 0.98410624],
       [0.01245246, 0.00297765, 0.98456985],
       [0.01256304, 0.00306055, 0.98437643],
       [0.01261937, 0.00301449, 0.9843662 ],
       [0.01272148, 0.00305904, 0.9842195 ],
       [0.01343279, 0.00316611, 0.9834011 ],
       [0.01348935, 0.00313343, 0.9833772 ],
       [0.01235128, 0.00300573, 0.984643  ],
       [0.

In [6]:
p_max = np.max(X)
train_gen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rescale=1/p_max,
                               brightness_range=[0.8, 1.0], zoom_range=[0.9, 1.0])
train_gen.fit(X_train)
val_gen = ImageDataGenerator(rescale=1/p_max)
val_gen.fit(X_test)

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit_generator(train_gen.flow(X_train, y_train, batch_size=50), 
          epochs=10, validation_data=val_gen.flow(X_test,y_test))

Epoch 1/10
23/23 [==============================] - 102s 4s/step - loss: 2.7572 - accuracy: 0.3875 - val_loss: 968623.2500 - val_accuracy: 0.3607
Epoch 2/10
23/23 [==============================] - 97s 4s/step - loss: 2.1460 - accuracy: 0.4161 - val_loss: 4751.5703 - val_accuracy: 0.3607
Epoch 3/10
23/23 [==============================] - 92s 4s/step - loss: 2.0889 - accuracy: 0.4518 - val_loss: 1750.6793 - val_accuracy: 0.2857
Epoch 4/10
23/23 [==============================] - 91s 4s/step - loss: 1.9117 - accuracy: 0.4688 - val_loss: 317125.4688 - val_accuracy: 0.2857
Epoch 5/10
23/23 [==============================] - 91s 4s/step - loss: 1.4659 - accuracy: 0.4500 - val_loss: 4.3472 - val_accuracy: 0.3607
Epoch 6/10
23/23 [==============================] - 89s 4s/step - loss: 1.6006 - accuracy: 0.4554 - val_loss: 1.0982 - val_accuracy: 0.3607
Epoch 7/10
23/23 [==============================] - 90s 4s/step - loss: 1.2980 - accuracy: 0.4964 - val_loss: 1.3412 - val_accuracy: 0.3607
Epo

In [3]:
ADNI = io.loadmat("ADNI_new.mat")
X, y = ADNI['X'], ADNI['y']
X = X / np.max(X)
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
IMSIZE = 100
input_layer = layers.Input([IMSIZE, IMSIZE, 30])

# Block 1
x = layers.Conv2D(32, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(input_layer)
x = layers.Conv2D(32, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(x)
x = layers.MaxPooling2D((3, 3), strides=(3, 3), name='block1_pool')(x)

# Block 2
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1')(x)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2')(x)
x = layers.MaxPooling2D((5, 5), strides=(3, 3), name='block2_pool')(x)
x = layers.BatchNormalization()(x)

# Block 3
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1')(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3')(x)
x = layers.MaxPooling2D((3, 3), strides=(3, 3), name='block3_pool')(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten(name='flatten')(x)
#x = layers.Dense(512, activation='relu', name='fc1')(x)
x = layers.Dense(3, activation='softmax', name='predictions')(x)
output_layer = x

model = Model(input_layer,output_layer)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 30)      0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 32)      8672      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 32)      9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 33, 33, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 33, 33, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 33, 33, 64)        36928     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 10, 10, 64)        0   

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs = 75, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/75
896/896 [==============================] - 12s 14ms/step - loss: 1.3930 - accuracy: 0.3382 - val_loss: 1.0965 - val_accuracy: 0.3527
Epoch 2/75
896/896 [==============================] - 13s 14ms/step - loss: 1.0645 - accuracy: 0.4420 - val_loss: 1.1098 - val_accuracy: 0.3348
Epoch 3/75
896/896 [==============================] - 13s 14ms/step - loss: 1.0140 - accuracy: 0.4844 - val_loss: 1.0948 - val_accuracy: 0.3438
Epoch 4/75
896/896 [==============================] - 13s 15ms/step - loss: 0.9251 - accuracy: 0.5424 - val_loss: 1.0792 - val_accuracy: 0.3884
Epoch 5/75
896/896 [==============================] - 13s 15ms/step - loss: 0.8560 - accuracy: 0.5893 - val_loss: 1.0899 - val_accuracy: 0.3839
Epoch 6/75
896/896 [==============================] - 13s 14ms/step - loss: 0.8091 - accuracy: 0.6384 - val_loss: 1.0711 - val_accuracy: 0.3884
Epoch 7/75
896/896 [==============================] - 12s 14ms/step - loss: 0.6404 - accur

896/896 [==============================] - 14s 15ms/step - loss: 2.1547e-04 - accuracy: 1.0000 - val_loss: 1.0677 - val_accuracy: 0.7009
Epoch 57/75
896/896 [==============================] - 13s 14ms/step - loss: 2.0308e-04 - accuracy: 1.0000 - val_loss: 1.0707 - val_accuracy: 0.7009
Epoch 58/75
896/896 [==============================] - 12s 14ms/step - loss: 1.9366e-04 - accuracy: 1.0000 - val_loss: 1.0740 - val_accuracy: 0.6964
Epoch 59/75
896/896 [==============================] - 12s 13ms/step - loss: 1.8680e-04 - accuracy: 1.0000 - val_loss: 1.0771 - val_accuracy: 0.7009
Epoch 60/75
896/896 [==============================] - 12s 13ms/step - loss: 1.9649e-04 - accuracy: 1.0000 - val_loss: 1.0794 - val_accuracy: 0.6920
Epoch 61/75
896/896 [==============================] - 12s 13ms/step - loss: 1.8021e-04 - accuracy: 1.0000 - val_loss: 1.0831 - val_accuracy: 0.7009
Epoch 62/75
896/896 [==============================] - 12s 13ms/step - loss: 1.8923e-04 - accuracy: 1.0000 - val_loss:

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs= 30, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/30
896/896 [==============================] - 12s 13ms/step - loss: 0.0536 - accuracy: 0.9844 - val_loss: 4.9997 - val_accuracy: 0.4286
Epoch 2/30
896/896 [==============================] - 11s 13ms/step - loss: 0.0824 - accuracy: 0.9710 - val_loss: 1.8705 - val_accuracy: 0.5938
Epoch 3/30
896/896 [==============================] - 11s 13ms/step - loss: 0.0682 - accuracy: 0.9799 - val_loss: 2.1197 - val_accuracy: 0.6339
Epoch 4/30
896/896 [==============================] - 12s 13ms/step - loss: 0.0608 - accuracy: 0.9777 - val_loss: 1.9094 - val_accuracy: 0.5714
Epoch 5/30
896/896 [==============================] - 11s 13ms/step - loss: 0.0250 - accuracy: 0.9944 - val_loss: 3.4339 - val_accuracy: 0.4598
Epoch 6/30
896/896 [==============================] - 11s 13ms/step - loss: 0.0213 - accuracy: 0.9955 - val_loss: 2.9574 - val_accuracy: 0.5089
Epoch 7/30
896/896 [==============================] - 11s 13ms/step - loss: 0.0066 - accur

In [16]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.6928571428571428

In [9]:
y_test_pred

array([[2.04988346e-01, 7.48854578e-01, 4.61571105e-02],
       [7.49915659e-01, 5.92215769e-02, 1.90862760e-01],
       [3.18640680e-03, 2.28562057e-02, 9.73957360e-01],
       [7.39132404e-01, 5.91316856e-02, 2.01735899e-01],
       [7.93632527e-04, 1.98452413e-01, 8.00753951e-01],
       [1.55496849e-02, 9.32507098e-01, 5.19433059e-02],
       [4.73990619e-01, 2.39930704e-01, 2.86078691e-01],
       [7.10559934e-02, 8.91954184e-01, 3.69898155e-02],
       [9.77606535e-01, 1.92658883e-02, 3.12758726e-03],
       [5.54366596e-03, 3.73650789e-01, 6.20805562e-01],
       [3.05737495e-01, 4.36177790e-01, 2.58084655e-01],
       [2.77307212e-01, 4.24099773e-01, 2.98593044e-01],
       [8.21426332e-01, 1.67468399e-01, 1.11053204e-02],
       [8.76179636e-02, 8.14302802e-01, 9.80791822e-02],
       [4.13177580e-01, 5.57622790e-01, 2.91996486e-02],
       [1.50120342e-02, 1.54098915e-02, 9.69578028e-01],
       [7.33003399e-05, 9.37161408e-03, 9.90555108e-01],
       [4.93476428e-02, 9.38062

In [6]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.6928571428571428

In [7]:
import h5py
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [3]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2 = filters

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2 = filters
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    shortcut = layers.Conv2D(filters2, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=3, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [5]:
IMSIZE = 100
input_layer = layers.Input([IMSIZE, IMSIZE, 30])

x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_layer)
x = layers.Conv2D(64, (7, 7),
                  strides=(2, 2),
                  padding='valid',
                  kernel_initializer='he_normal',
                  name='conv1')(x)
x = layers.BatchNormalization(axis=3, name='bn_conv1')(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, 3, [64, 64], stage=2, block='a', strides=(1, 1))
x = identity_block(x, 3, [64, 64], stage=2, block='b')
x = identity_block(x, 3, [64, 64], stage=2, block='c')

x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
output_layer = layers.Dense(3, activation='softmax', name='fc1000')(x)

model = Model(input_layer,output_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 100, 30) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 30) 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   94144       conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs=25, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/25
896/896 [==============================] - 36s 40ms/step - loss: 1.2124 - accuracy: 0.4330 - val_loss: 1.3171 - val_accuracy: 0.3304
Epoch 2/25
896/896 [==============================] - 32s 36ms/step - loss: 1.0159 - accuracy: 0.4565 - val_loss: 1.4262 - val_accuracy: 0.3884
Epoch 3/25
896/896 [==============================] - 30s 33ms/step - loss: 0.9616 - accuracy: 0.5268 - val_loss: 1.2381 - val_accuracy: 0.3795
Epoch 4/25
896/896 [==============================] - 33s 37ms/step - loss: 0.9086 - accuracy: 0.5558 - val_loss: 1.2992 - val_accuracy: 0.3616
Epoch 5/25
896/896 [==============================] - 31s 34ms/step - loss: 0.8664 - accuracy: 0.5982 - val_loss: 1.2225 - val_accuracy: 0.3839
Epoch 6/25
896/896 [==============================] - 31s 35ms/step - loss: 0.8586 - accuracy: 0.5893 - val_loss: 1.4035 - val_accuracy: 0.3571
Epoch 7/25
896/896 [==============================] - 33s 37ms/step - loss: 0.7931 - accur

In [7]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.425

In [8]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2 = filters

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      activation='relu',
                      name=conv_name_base + '2a')(input_tensor)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.add([x, input_tensor])
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2 = filters
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal', activation='relu',
                      name=conv_name_base + '2a')(input_tensor)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal', activation='relu',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(x)

    shortcut = layers.Conv2D(filters2, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [11]:
IMSIZE = 100
input_layer = layers.Input([IMSIZE, IMSIZE, 30])

x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_layer)
x = layers.Conv2D(64, (7, 7),
                  strides=(2, 2),
                  padding='valid',
                  activation='relu',
                  kernel_initializer='he_normal',
                  name='conv1')(x)
x = layers.BatchNormalization(axis=3, name='bn_conv1')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, 3, [64, 128], stage=2, block='a', strides=(1, 1))
x = identity_block(x, 3, [64, 128], stage=2, block='b')
x = identity_block(x, 3, [64, 128], stage=2, block='c')

x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
output_layer = layers.Dense(3, activation='softmax', name='fc1000')(x)

model = Model(input_layer,output_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 100, 100, 30) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 30) 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   94144       conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs=25, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/25
896/896 [==============================] - 31s 35ms/step - loss: 1.3336 - accuracy: 0.4040 - val_loss: 3.5482 - val_accuracy: 0.3304
Epoch 2/25
896/896 [==============================] - 32s 36ms/step - loss: 1.0222 - accuracy: 0.4587 - val_loss: 1.5987 - val_accuracy: 0.3304
Epoch 3/25
896/896 [==============================] - 34s 38ms/step - loss: 0.9808 - accuracy: 0.5167 - val_loss: 1.2268 - val_accuracy: 0.3929
Epoch 4/25
896/896 [==============================] - 35s 39ms/step - loss: 0.9508 - accuracy: 0.5424 - val_loss: 2.0290 - val_accuracy: 0.3304
Epoch 5/25
896/896 [==============================] - 35s 39ms/step - loss: 0.9353 - accuracy: 0.5324 - val_loss: 1.6030 - val_accuracy: 0.3348
Epoch 6/25
896/896 [==============================] - 35s 39ms/step - loss: 0.8659 - accuracy: 0.5792 - val_loss: 1.5233 - val_accuracy: 0.3348
Epoch 7/25
896/896 [==============================] - 36s 40ms/step - loss: 0.7989 - accur

In [13]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.5642857142857143

In [18]:
np.unique(np.argmax(y_test, axis=1), return_counts=True)

(array([0, 1, 2]), array([ 80,  99, 101]))

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, 
          epochs=40, validation_split=0.2)

Train on 896 samples, validate on 224 samples
Epoch 1/40
896/896 [==============================] - 32s 35ms/step - loss: 0.0502 - accuracy: 0.9955 - val_loss: 2.0125 - val_accuracy: 0.4375
Epoch 2/40
896/896 [==============================] - 32s 36ms/step - loss: 0.0668 - accuracy: 0.9844 - val_loss: 5.0231 - val_accuracy: 0.3750
Epoch 3/40
896/896 [==============================] - 35s 39ms/step - loss: 0.0689 - accuracy: 0.9777 - val_loss: 8.2067 - val_accuracy: 0.3304
Epoch 4/40
896/896 [==============================] - 36s 40ms/step - loss: 0.0665 - accuracy: 0.9877 - val_loss: 3.0719 - val_accuracy: 0.4107
Epoch 5/40
896/896 [==============================] - 36s 40ms/step - loss: 0.0505 - accuracy: 0.9911 - val_loss: 1.8194 - val_accuracy: 0.4732
Epoch 6/40
896/896 [==============================] - 37s 41ms/step - loss: 0.0280 - accuracy: 0.9978 - val_loss: 3.1540 - val_accuracy: 0.4107
Epoch 7/40
896/896 [==============================] - 37s 41ms/step - loss: 0.0219 - accur

In [20]:
y_test_pred = model.predict(X_test)
np.mean(np.argmax(y_test, axis=1) == np.argmax(y_test_pred, axis=1))

0.6535714285714286

In [21]:
y_test_pred

array([[1.33089386e-02, 9.78519261e-01, 8.17178376e-03],
       [5.87763369e-01, 3.34178835e-01, 7.80578405e-02],
       [4.28377498e-05, 7.23131076e-02, 9.27644014e-01],
       [9.93621051e-01, 5.88057376e-03, 4.98277601e-04],
       [6.51015714e-02, 7.86492407e-01, 1.48406044e-01],
       [2.78795838e-01, 1.05335392e-01, 6.15868747e-01],
       [3.47530276e-01, 1.45680517e-01, 5.06789148e-01],
       [2.75418401e-01, 7.23439455e-01, 1.14220742e-03],
       [9.94773924e-01, 4.66954429e-03, 5.56504296e-04],
       [2.97659775e-04, 9.48392153e-01, 5.13101518e-02],
       [4.55379905e-03, 9.95314956e-01, 1.31253575e-04],
       [7.02044636e-04, 8.78735408e-02, 9.11424398e-01],
       [9.88217056e-01, 1.05651850e-02, 1.21778331e-03],
       [2.10188683e-02, 9.78770673e-01, 2.10445054e-04],
       [8.73447120e-01, 3.33010466e-05, 1.26519561e-01],
       [2.04733922e-04, 1.13005197e-04, 9.99682307e-01],
       [1.89436381e-04, 1.11322394e-02, 9.88678277e-01],
       [3.61694751e-04, 9.99360